In [154]:
import pandas as pd
import json

In [155]:
file = open('input_file.json')

In [156]:
a = json.load(file)

In [157]:
data_dc = a

In [158]:
data_dc

{'Starbucks': {'Name': 'Starbucks',
  'Dishes': [{'name': 'Caffe Americano',
    'description': 'Espresso shots topped with hot water create a light layer of crema culminating in this wonderfully rich cup with depth and nuance. Pro tip: For an additional boost, ask to try this with an extra shot.',
    'imageLink': 'https://media-cdn.grubhub.com/image/upload/w_1.46,h_1,c_lpad/tapingo-shops-assets/media/images/941/bcd/a4f817112069a2a985978813eeef447702',
    'price': 345,
    'category': 'Hot Coffees'},
   {'name': 'Caffe Misto',
    'description': "A 1-to-1 combination of fresh-brewed coffee and steamed milk add up to one distinctly delicious coffee drink that's remarkably mixed.",
    'imageLink': 'https://media-cdn.grubhub.com/image/upload/w_1.46,h_1,c_lpad/tapingo-shops-assets/media/images/e12/006/f23626ac0f0674d213edd6492052f02149',
    'price': 425,
    'category': 'Hot Coffees'},
   {'name': 'Blonde Roast Coffee',
    'description': "Lightly roasted coffee that's soft, mellow and

In [162]:
type(data_dc['Starbucks']['Dishes'][0]['name'])

str

In [164]:
output = []
for key in data_dc:
    store_name = key
    store_location = data_dc[key]['Location']
    for index in data_dc[key]['Dishes']:
        # dish_name = data_dc[key]['Dishes'][index]['name']
        # dish_desc = data_dc[key]['Dishes'][index]['description']
        # dish_img = data_dc[key]['Dishes'][index]['imageLink']
        # dish_price = data_dc[key]['Dishes'][index]['price']
        # dish_category = data_dc[key]['Dishes'][index]['category']
        output.append({
            'shop_name': store_name,
            'location': store_location,
            'dish_name': index['name'],
            'dish_description': index['description'],
            'dish_image_link': index['imageLink'],
            'dish_price': index['price'],
            'dish_category': index['category']
        })

len(output)

262

In [166]:
with open('input_file2.json', 'w') as w:
    json.dump(output, w, indent = 4)